In [2]:
import pandas as pd
import numpy as np

In [3]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_parquet('data/sample.parquet')
df

,time,value,field,robot_id,run_uuid,sensor_type
0,2022-11-23T20:40:00.005Z,821.780800,x,1,8.910096e+18,encoder
1,2022-11-23T20:40:00.017Z,821.821700,x,1,8.910096e+18,encoder
2,2022-11-23T20:40:00.029Z,821.850700,x,1,8.910096e+18,encoder
3,2022-11-23T20:40:00.041Z,821.896400,x,1,8.910096e+18,encoder
4,2022-11-23T20:40:00.053Z,821.957300,x,1,8.910096e+18,encoder
...,...,...,...,...,...,...
1546586,2022-11-23T20:41:17.59Z,-85.692373,fx,1,1.240519e+19,load_cell
1546587,2022-11-23T20:41:17.6Z,-87.231436,fx,1,1.240519e+19,load_cell
1546588,2022-11-23T20:41:17.61Z,-85.649405,fx,1,1.240519e+19,load_cell
1546589,2022-11-23T20:41:17.62Z,-86.430655,fx,1,1.240519e+19,load_cell


## Sort the rows based on the time to make sure that the rows are stored sequentialy

In [5]:
df = df.sort_values('time')
df

,time,value,field,robot_id,run_uuid,sensor_type
878828,2022-11-23T20:40:00.001Z,716.528276,fy,1,7.582293e+18,load_cell
938829,2022-11-23T20:40:00.001Z,-1547.340972,fz,1,7.582293e+18,load_cell
818827,2022-11-23T20:40:00.001Z,-1192.046953,fx,1,7.582293e+18,load_cell
1118830,2022-11-23T20:40:00.003Z,84.484822,fz,2,7.582293e+18,load_cell
1058830,2022-11-23T20:40:00.003Z,489.207227,fy,2,7.582293e+18,load_cell
...,...,...,...,...,...,...
550011,2022-11-23T20:49:59.9Z,1000.769000,y,2,7.582293e+18,encoder
600012,2022-11-23T20:49:59.9Z,-771.632000,z,2,7.582293e+18,encoder
549936,2022-11-23T20:49:59Z,1000.769000,y,2,7.582293e+18,encoder
599937,2022-11-23T20:49:59Z,-771.632000,z,2,7.582293e+18,encoder


# 2.1 Preprocess and Clean

## A method to check the data type of each column. If all column types are as expected this method pass otherwise it shows the columns with a mismatch

In [6]:
def check_data_types(df, expected_types):
    """
    Check if the DataFrame's columns have the expected data types.

    :param df: pandas DataFrame to check
    :param expected_types: Dictionary of expected data types {column_name: expected_type}
    :return: None if types match, otherwise raises ValueError with mismatch details
    """
    for column, expected_type in expected_types.items():
        if column in df.columns:
            if not pd.api.types.is_dtype_equal(df[column].dtype, expected_type):
                raise ValueError(f"Data type mismatch in column '{column}'. Expected {expected_type}, found {df[column].dtype}")
        else:
            raise ValueError(f"Column '{column}' not found in DataFrame")
    print("All column data types are as expected.")

In [7]:
# Define the expected types
expected_types = {
    'time': 'object',
    'value': 'float64',
    'field': 'object',
    'robot_id': 'int64',
    'run_uuid': 'float64',
    'sensor_type': 'object'
}

# Check data types
check_data_types(df, expected_types)

All column data types are as expected.


In [6]:
df.isnull().sum()

time           0
value          0
field          0
robot_id       0
run_uuid       0
sensor_type    0
dtype: int64

## The data doesn't have missing data. However in case of missing data, we can use imputation to replace mode or most frequent value in a column with the null values. If the number of null values is small, we can remove the rows containing null values. It depands on the data to choose the method, since I don't know how much missing value we might have.

# 2.2 Convert timeseries to a wide format

## Here, I split the data based on each part (run_uuid)

In [7]:
# Identify Unique run_uuid Values
unique_run_uuids = df['run_uuid'].unique()

# Split the DataFrame
dfs = {}
for run_uuid in unique_run_uuids:
    # Create a separate DataFrame for each run_uuid
    dfs[run_uuid] = df[df['run_uuid'] == run_uuid]

## For each part, I concatenated the field an robot_id and then used pivot to have them in one row and convert them to a wide format.

In [8]:
for run_uuid, temp_df in dfs.items():
    # Explicitly create a copy of the DataFrame slice
    temp_df = temp_df.copy()

    # Combine 'field' and 'robot_id'
    temp_df['field_robot'] = temp_df['field'] + '_' + temp_df['robot_id'].astype(str)

    # Pivot the DataFrame
    pivot_df = temp_df.pivot_table(index='time', columns='field_robot', values='value', aggfunc='first')
    
    # Replace the original DataFrame in the dictionary with the pivoted one
    dfs[run_uuid] = pivot_df

## Here for each part, there is one dataframe. They have all possible columns, but for some of the columns there is no data. So I keep the columns and add '0' for such columns.

In [9]:
required_columns = ['fx_1', 'fx_2', 'fy_1', 'fy_2', 'fz_1', 'fz_2', 'x_1', 'x_2', 'y_1', 'y_2', 'z_1', 'z_2']

for run_uuid, df in dfs.items():
    # Identify missing columns
    missing_columns = [col for col in required_columns if col not in df.columns]

    # Add missing columns with default value of 0
    for col in missing_columns:
        df[col] = 0

    # Optionally, reorder columns to match the order in required_columns
    df = df.reindex(columns=required_columns)

    # Update the DataFrame in the dictionary
    dfs[run_uuid] = df

## After converting the timeseries to individual features, there are many gaps in the data. So the dataframe has null values. I used interpolation to fill the null values. Since the data is a time series and there is some form of continutiy, interpolation could be used here. It estimated the null values using existing values.

In [10]:
for run_uuid, df in dfs.items():
    # Interpolate to fill missing values
    df_interpolated = df.interpolate()

    # Forward fill or backward fill to handle remaining missing values
    df_filled = df_interpolated.fillna(method='bfill')

    # Update the DataFrame in the dictionary
    dfs[run_uuid] = df_filled

# 2.3 Include Engineered/Calculated Features

## The velocity, acceleration, total velocity, total acceleration and total force values are calculated here. I used the current position of the robot, the previous position and the time difference between them to calcualte teh velocity. The way that the velocity and acceleration might need modifications but the logic works fine here.

In [11]:
for run_uuid, df in dfs.items():
    # Ensure the index is a datetime index
    df.index = pd.to_datetime(df.index)

    # Calculate the time differences in seconds
    df['time_diff'] = df.index.to_series().diff().dt.total_seconds()

    # Calculate velocities
    for axis in ['x', 'y', 'z']:
        for robot_id in [1, 2]:
            pos_col = f'{axis}_{robot_id}'
            vel_col = f'v{axis}_{robot_id}'
            df[vel_col] = df[pos_col].diff() / df['time_diff']

    # Calculate accelerations
    for axis in ['x', 'y', 'z']:
        for robot_id in [1, 2]:
            vel_col = f'v{axis}_{robot_id}'
            acc_col = f'a{axis}_{robot_id}'
            df[acc_col] = df[vel_col].diff() / df['time_diff']

    # Calculate total velocities and accelerations
    for robot_id in [1, 2]:
        df[f'v{robot_id}'] = (df[f'vx_{robot_id}']**2 + df[f'vy_{robot_id}']**2 + df[f'vz_{robot_id}']**2)**0.5
        df[f'a{robot_id}'] = (df[f'ax_{robot_id}']**2 + df[f'ay_{robot_id}']**2 + df[f'az_{robot_id}']**2)**0.5

    # Calculate total forces
    for robot_id in [1, 2]:
        df[f'f{robot_id}'] = abs(df[f'fx_{robot_id}']) + abs(df[f'fy_{robot_id}']) + abs(df[f'fz_{robot_id}'])

    # Update the DataFrame in the dictionary
    dfs[run_uuid] = df

## In the following dataframes, you can see the transformed features for each part

In [13]:
dfs[7.58229308099147e+18]

field_robot,fx_1,fx_2,fy_1,fy_2,fz_1,fz_2,x_1,x_2,y_1,y_2,z_1,z_2,time_diff,vx_1,vx_2,vy_1,vy_2,vz_1,vz_2,ax_1,ax_2,ay_1,ay_2,az_1,az_2,v1,a1,v2,a2,f1,f2
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-23 20:40:00.001000+00:00,-1192.046953,-546.669903,716.528276,489.207227,-1547.340972,84.484822,2862.302,3050.773,995.860000,1000.769,-48.5650,-771.632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3455.916201,1120.361952
2022-11-23 20:40:00.003000+00:00,-1192.611162,-546.669903,716.018389,489.207227,-1547.325957,84.484822,2862.302,3050.773,995.860000,1000.769,-48.5650,-771.632,0.002,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.0,NaN,3455.955508,1120.361952
2022-11-23 20:40:00.008000+00:00,-1193.175371,-547.196438,715.508501,489.229962,-1547.310942,83.530789,2862.302,3050.773,995.860000,1000.769,-48.5650,-771.632,0.005,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,3455.994814,1119.957189
2022-11-23 20:40:00.009000+00:00,-1193.739580,-547.722973,714.998613,489.252698,-1547.295928,82.576756,2862.302,3050.773,995.863167,1000.769,-48.5655,-771.632,0.001,0.000000,0.0,3.166667,0.0,-0.500000,0.0,0.000000,0.0,3166.666667,0.0,-500.000000,0.0,3.205897,3205.897344,0.0,0.0,3456.034121,1119.552427
2022-11-23 20:40:00.011000+00:00,-1194.303789,-548.249509,714.488726,489.275433,-1547.280913,81.622723,2862.302,3050.773,995.866333,1000.769,-48.5660,-771.632,0.002,0.000000,0.0,1.583333,0.0,-0.250000,0.0,0.000000,0.0,-791.666667,0.0,125.000000,0.0,1.602949,801.474336,0.0,0.0,3456.073428,1119.147665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23 20:49:59.994000+00:00,-1383.952796,-547.053387,-198.234419,489.750887,-1414.971587,83.226399,3064.177,3050.773,1225.877000,1000.769,-57.5040,-771.632,0.006,52.444444,0.0,3.611111,0.0,0.166667,0.0,-4370.370370,0.0,-300.925926,0.0,-13.888889,0.0,52.568885,4380.740404,0.0,0.0,2997.158802,1120.030673
2022-11-23 20:49:59.996000+00:00,-1379.871579,-546.849286,-203.198134,489.452873,-1401.572051,83.235981,3064.177,3050.773,1225.877000,1000.769,-57.5040,-771.632,0.002,0.000000,0.0,0.000000,0.0,0.000000,0.0,-26222.222222,0.0,-1805.555556,0.0,-83.333333,0.0,0.000000,26284.442425,0.0,0.0,2984.641763,1119.538140
2022-11-23 20:49:59.998000+00:00,-1375.790361,-546.645184,-208.161848,489.154858,-1388.172515,83.245564,3064.177,3050.773,1225.877000,1000.769,-57.5040,-771.632,0.002,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,2972.124724,1119.045607


In [14]:
dfs[1.2405186538561671e+19]

field_robot,fx_1,fx_2,fy_1,fy_2,fz_1,fz_2,x_1,x_2,y_1,y_2,z_1,z_2,time_diff,vx_1,vx_2,vy_1,vy_2,vz_1,vz_2,ax_1,ax_2,ay_1,ay_2,az_1,az_2,v1,a1,v2,a2,f1,f2
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-23 20:40:00.003000+00:00,-88.747061,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.747061,0
2022-11-23 20:40:00.013000+00:00,-91.774405,0,0,0,0,0,0,0,0,0,0,0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,91.774405,0
2022-11-23 20:40:00.023000+00:00,-90.149405,0,0,0,0,0,0,0,0,0,0,0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.149405,0
2022-11-23 20:40:00.033000+00:00,-85.747061,0,0,0,0,0,0,0,0,0,0,0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.747061,0
2022-11-23 20:40:00.043000+00:00,-90.184561,0,0,0,0,0,0,0,0,0,0,0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.184561,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23 20:41:17.610000+00:00,-85.649405,0,0,0,0,0,0,0,0,0,0,0,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.649405,0
2022-11-23 20:41:17.620000+00:00,-86.430655,0,0,0,0,0,0,0,0,0,0,0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.430655,0
2022-11-23 20:41:17.630000+00:00,-86.477530,0,0,0,0,0,0,0,0,0,0,0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.477530,0


In [15]:
dfs[8.910095844186657e+18]

field_robot,fx_1,fx_2,fy_1,fy_2,fz_1,fz_2,x_1,x_2,y_1,y_2,z_1,z_2,time_diff,vx_1,vx_2,vy_1,vy_2,vz_1,vz_2,ax_1,ax_2,ay_1,ay_2,az_1,az_2,v1,a1,v2,a2,f1,f2
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-23 20:40:00.005000+00:00,0,0,0,0,0,0,821.780800,823.290600,326.525600,320.510300,-1.80510,-2.910100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2022-11-23 20:40:00.008000+00:00,0,0,0,0,0,0,821.801250,823.290600,326.534300,320.510300,-1.80535,-2.910100,0.003,6.816667,0.000000,2.900000,0.000000,-0.083333,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,7.408366,NaN,0.000000,NaN,0,0
2022-11-23 20:40:00.017000+00:00,0,0,0,0,0,0,821.821700,823.307533,326.543000,320.514333,-1.80560,-2.912167,0.009,2.272222,1.881481,0.966667,0.448148,-0.027778,-0.229630,-504.938272,209.053498,-214.814815,49.794239,6.172840,-25.514403,2.469455,548.767862,1.947701,216.411220,0,0
2022-11-23 20:40:00.029000+00:00,0,0,0,0,0,0,821.850700,823.324467,326.558300,320.518367,-1.81380,-2.914233,0.012,2.416667,1.411111,1.275000,0.336111,-0.683333,-0.172222,12.037037,-39.197531,25.694444,-9.336420,-54.629630,4.783951,2.816531,61.558843,1.460776,40.577104,0,0
2022-11-23 20:40:00.020000+00:00,0,0,0,0,0,0,821.865933,823.341400,326.562833,320.522400,-1.81160,-2.916300,-0.009,-1.692593,-1.881481,-0.503704,-0.448148,-0.244444,0.229630,456.584362,365.843621,197.633745,87.139918,-48.765432,-44.650206,1.782790,499.906435,1.947701,378.719634,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23 20:49:59.975000+00:00,0,0,0,0,0,0,809.291250,810.446700,333.509000,322.154300,-12.78915,-8.372600,0.010,-42.345000,-47.030000,-7.110000,-7.995000,0.305000,0.195000,16630.500000,18812.000000,2686.500000,3198.000000,98.000000,-78.000000,42.938842,16846.376955,47.705125,19082.049995,0,0
2022-11-23 20:49:59.977000+00:00,0,0,0,0,0,0,808.867800,809.985300,333.437900,322.084550,-12.78610,-8.377850,0.002,-211.725000,-230.700000,-35.550000,-34.875000,1.525000,-2.625000,-84690.000000,-91835.000000,-14220.000000,-13440.000000,610.000000,-1410.000000,214.694210,85877.683946,233.335909,92823.967406,0,0
2022-11-23 20:49:59.987000+00:00,0,0,0,0,0,0,808.450350,809.523900,333.371050,322.014800,-12.78010,-8.383100,0.010,-41.745000,-46.140000,-6.685000,-6.975000,0.600000,-0.525000,16998.000000,18456.000000,2886.500000,2790.000000,-92.500000,210.000000,42.281133,17241.590486,46.667182,18666.872689,0,0


In [16]:
dfs[6.176976534744076e+18]

field_robot,fx_1,fx_2,fy_1,fy_2,fz_1,fz_2,x_1,x_2,y_1,y_2,z_1,z_2,time_diff,vx_1,vx_2,vy_1,vy_2,vz_1,vz_2,ax_1,ax_2,ay_1,ay_2,az_1,az_2,v1,a1,v2,a2,f1,f2
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-23 20:40:00.007000+00:00,176.096381,50.373027,174.268623,-416.060405,-258.179417,80.691381,1440.79,1438.412,936.925,939.383,-222.290000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,608.544421,547.124813
2022-11-23 20:40:00.008000+00:00,176.882016,50.373027,173.625221,-416.060405,-258.608595,80.691381,1440.79,1438.412,936.925,939.383,-222.290000,0,0.001,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.0,NaN,609.115832,547.124813
2022-11-23 20:40:00.011000+00:00,177.667650,50.373027,172.981819,-416.060405,-259.037774,80.691381,1440.79,1438.412,936.925,939.383,-222.290000,0,0.003,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.0,609.687243,547.124813
2022-11-23 20:40:00.017000+00:00,178.453284,50.373027,172.338417,-416.060405,-259.466953,80.691381,1440.79,1438.412,936.925,939.383,-222.290000,0,0.006,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.0,610.258655,547.124813
2022-11-23 20:40:00.010000+00:00,178.242767,50.373027,172.583463,-416.060405,-258.628656,80.691381,1440.79,1438.412,936.925,939.383,-222.290000,0,-0.007,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.0,609.454885,547.124813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23 20:49:59.986000+00:00,178.453284,52.317689,172.338417,-415.717754,-259.466953,79.025467,1440.79,1438.412,936.925,939.383,-222.289200,0,0.003,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,-1.111111e+01,0.0,0.066667,1.111111e+01,0.0,0.0,610.258655,547.060909
2022-11-23 20:49:59.987000+00:00,178.089299,52.152579,172.333804,-415.398174,-259.464512,78.592005,1440.79,1438.412,936.925,939.383,-222.289000,0,0.001,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,1.333333e+02,0.0,0.200000,1.333333e+02,0.0,0.0,609.887614,546.142758
2022-11-23 20:49:59.993000+00:00,177.725313,51.987468,172.329191,-415.078594,-259.462070,78.158544,1440.79,1438.412,936.925,939.383,-222.289333,0,0.006,0.0,0.0,0.0,0.0,-0.055556,0.0,0.0,0.0,0.0,0.0,-4.259259e+01,0.0,0.055556,4.259259e+01,0.0,0.0,609.516574,545.224606


# 2.4 Calculate Runtime Statistics

## Runtime Statistics is produced for each part and stored in a pandas dataframe. This dataframe is then saved in an csv file.

In [18]:
results_list = []

for run_uuid, df in dfs.items():
    # Ensure the index is a datetime index
    df.index = pd.to_datetime(df.index)

    # Start and stop times
    start_time = df.index[0]
    stop_time = df.index[-1]

    # Total runtime
    total_runtime = stop_time - start_time

    # Initialize total distance for each robot
    total_distance = {1: 0, 2: 0}

    # Calculate total distance traveled for each robot
    for robot_id in [1, 2]:
        # Calculate differences in x, y, z coordinates
        dx = df[f'x_{robot_id}'].diff().fillna(0)
        dy = df[f'y_{robot_id}'].diff().fillna(0)
        dz = df[f'z_{robot_id}'].diff().fillna(0)

        # Calculate distance for each step and sum
        total_distance[robot_id] = np.sqrt(dx**2 + dy**2 + dz**2).sum()

    # Create a dictionary of the results and add it to the list
    results_dict = {
        'run_uuid': run_uuid,
        'start_time': start_time,
        'stop_time': stop_time,
        'total_runtime': total_runtime,
        'total_distance_1': total_distance[1],
        'total_distance_2': total_distance[2]
    }
    results_list.append(results_dict)

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results_list)

In [19]:
results_df

,run_uuid,start_time,stop_time,total_runtime,total_distance_1,total_distance_2
0,7.582293e+18,2022-11-23 20:40:00.001000+00:00,2022-11-23 20:49:59+00:00,0 days 00:09:58.999000,52270.618331,24.307657
1,1.240519e+19,2022-11-23 20:40:00.003000+00:00,2022-11-23 20:41:17+00:00,0 days 00:01:16.997000,0.000000,0.000000
2,8.910096e+18,2022-11-23 20:40:00.005000+00:00,2022-11-23 20:49:59.999000+00:00,0 days 00:09:59.994000,36335.283037,41086.188758
3,6.176977e+18,2022-11-23 20:40:00.007000+00:00,2022-11-23 20:49:59.999000+00:00,0 days 00:09:59.992000,10.466115,7.210912


In [14]:
# Specify the file name for the CSV file
csv_file_name = 'results.csv'

# Write the DataFrame to a CSV file
results_df.to_csv(csv_file_name, index=False)